In [32]:
import functools
#下面要使用完整的修饰器定义 所准备

In [2]:
#求和函数 可变参数
def cal_sum(*args):
	ax = 0
	
	for n in args:
		ax = ax + n
	
	return ax

print(cal_sum(10,12,4234,324))

4580


In [3]:
#函数式编程
#函数作为返回值

def lazy_sum(*args):
    this = 52 #这个变量和lazy_sum的参数 在下面的sum函数都可见
    #都保存在返回的函数中 称为“闭包”
	def sum():
		ax = 0
		for n in args:
			ax = ax + n
		
		return ax
	
	return sum

#f是一个函数
f = lazy_sum(1,2,5,2)
print(f)

<function lazy_sum.<locals>.sum at 0x00000191FF93A730>


In [4]:
f()

10

In [5]:
f1 = lazy_sum(1,2,5,4,8,21)
f2 = lazy_sum(1,2,5,4,8,21)
#是不相等的 即便参数相同
print(f1==f2)

False


In [6]:
def count():
    fs = []
    for i in range(1 , 4):
        def f():
            return i*i
        
        fs.append(f)
    
    return fs

f1,f2,f3 = count()

In [8]:
print(f1())
print(f2())
print(f3())
#原因就在于返回的函数引用了变量i，但它并非立刻执行。
#等到3个函数都返回时，
#它们所引用的变量i已经变成了3，因此最终结果为9
#返回闭包返回函数不用引用任何循环变量 或者后续会发生变化的变量

9
9
9


In [9]:
#如果非要使用循环变量 创建一个函数 用该函数的参数绑定循环变量当前的值
def count2():
    def f(j):
        def g():
            return j*j
        return g
    
    fs = []
    #使用立即执行的方式 直接计算了
    for i in range(1 , 4):
        fs.append(f(i))
        
    return fs

In [11]:
f1 , f2 , f3 = count2()
print(f1())
print(f2())
print(f3())


1
4
9


In [12]:
#匿名函数 不需要显示地的定义函数
#关键字 lambda 表示匿名函数 
a = list(map(lambda x:x*x , [1,2,3,4,5]))
print(a)

[1, 4, 9, 16, 25]


In [13]:
lambda x:x*x
#上下两种方式一样
def f(x):
    return x*x

In [15]:
#匿名函数也是一个函数对象 也可以把匿名函数赋值给一个变量 
#再利用该变量调用函数

f = lambda x:x*x
print(f(2))

4


In [16]:
#也可以把匿名函数返回回来
def build(x,y):
    return lambda :x*x+y*y

kk = build(1,2)
print(kk)
print(kk())

<function build.<locals>.<lambda> at 0x00000191FF93AB70>
5


In [17]:
#感觉返回函数 像是返回函数指针

In [26]:
#==================
#==================
#装饰器
#函数是对象

#把@log放到now()函数的定义处，相当于执行了语句：
#now = log(now)
'''
由于log()是一个decorator，返回一个函数，所以，
原来的now()函数仍然存在，只是现在同名的now变量指向了新的函数，
于是调用now()将执行新函数，即在log()函数中返回的wrapper()函数。
wrapper()函数的参数定义是(*args, **kw)，
因此，wrapper()函数可以接受任意参数的调用。
在wrapper()函数内，首先打印日志，再紧接着调用原始函数。
'''
#log()就是一个装饰器

@log
def now():
    print('haha')

#f = now
#f()
now()

call now()
haha


In [23]:
#函数对象有一个__name__属性 可以拿到函数的名字
print(now.__name__)
print(f.__name__)

now
now


In [24]:
def log(func):
    def wrapper(*args , **kw):
        print('call %s()' % func.__name__)
        return func(*args , **kw)
    
    return wrapper

In [31]:
def log2(text):
    def decorator(func):
        def wrapper(*args , **kw):
            print('%s %s' % (text , func.__name__))
            return func(*args , **kw)
        return wrapper
    return decorator

#now = log('execute')(now) 实际调用情况
@log2('zheli')
def now2():
    print('fd')

now2()
print(now2.__name__)

'''
首先执行log('execute')，返回的是decorator函数，
再调用返回的函数，参数是now函数，返回值最终是wrapper函数
'''
'''
不需要编写wrapper.__name__ = func.__name__这样的代码，
Python内置的functools.wraps就是干这个事的，'''
'''
因为返回的那个wrapper()函数名字就是'wrapper'，所以，
需要把原始函数的__name__等属性复制到wrapper()函数中，
否则，有些依赖函数签名的代码执行就会出错。
'''


zheli now2
fd
wrapper


In [33]:
#无参数修饰器
#需要引入functools

def log_t(func):
    @functools.wraps(func)
    def wrapper(*args , **kw):
        print('call %s():' % func.__name__)
        return func(*args , **kw)
    return wrapper

In [34]:
#无参数修饰器
def log(text):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args , **kw):
            print('%s %s():' % (text , func.__name__))
            return func(*args , **kw)
        return wrapper
    return decorator

In [36]:
#==============
#==============
#偏函数

#字符串转为数字 10进制
number = int('12' , base = 10)
print(number)
#8进制
number = int('12' , base = 8)
print(number)

12
10


In [38]:
#2进制
def int2(x , base = 2):
    return int(x , base)

number = int2('10010')
print(number)

18


In [40]:
#引入偏函数 需要引入functools
int2 = functools.partial(int , base = 2)
number = int2('10')
print(number)

'''
所以，简单总结functools.partial的作用就是，
把一个函数的某些参数给固定住（也就是设置默认值），
返回一个新的函数，调用这个新函数会更简单。
注意到上面的新的int2函数，仅仅是把base参数重新设定默认值为2，
但也可以在函数调用时传入其他值
'''

#下面调用也正确
number = int2('10' , base = 8)
print(number)

'''
最后，创建偏函数时，实际上可以接收 函数对象、*args和**kw这3个参数，当传入：
'''
#int('10010)
#相当于
#kw = { 'base': 2 }
#int('10010', **kw)

2
8


In [41]:
#下面代码
#实际上会把10作为*args的一部分自动加到左边，也就是
max2 = functools.partial(max , 10)
result = max2(5,6,7)

print(result)

10


In [42]:
#上述代码块 实际如下
args = (10 , 5 , 6 ,7)
result = max(*args)
print(result)

10


In [ ]:
#当函数的参数个数太多，需要简化时，
#使用functools.partial可以创建一个新的函数，
#这个新函数可以固定住原函数的部分参数，从而在调用时更简单。